In [29]:
from ctypes import *
import time
from sys import path
from os import sep
import numpy as np

dwf = cdll.LoadLibrary("libdwf.so")
constants_path = "/usr/share/digilent/waveforms/samples/py"
path.append(constants_path)
import dwfconstants as constants

In [30]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0)) #setting VDD = 3V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +3.01V


In [31]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [32]:
write_register(ENABLE_REG, 0x01) #Power on / PON = 1
time.sleep(0.01)
write_register(ENABLE_REG, 0x03) #Now, also RGBC enable / PON = 1, AEN = 1
write_register(ATIME_REG, 0xF6) #To run for 24ms, upto a max count of 10,240
write_register(CONTROL_REG, 0x02) #Setting AGAIN=16

True

In [33]:
#Function to test the sensor output
def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [34]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)

In [35]:
cleanup()
#call it everytime after work is done

In [17]:
write_register(CONTROL_REG,0)
for i in range(256):
    write_register(ATIME_REG,255-i)
    colors = read_colors()

    print(f"Clear: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

Clear: 2, R: 1, G: 1, B: 1
Clear: 4, R: 1, G: 2, B: 1
Clear: 6, R: 2, G: 2, B: 2
Clear: 8, R: 3, G: 3, B: 3
Clear: 10, R: 4, G: 4, B: 3
Clear: 12, R: 4, G: 5, B: 4
Clear: 14, R: 5, G: 5, B: 5
Clear: 16, R: 6, G: 6, B: 5
Clear: 18, R: 6, G: 7, B: 6
Clear: 20, R: 7, G: 8, B: 7
Clear: 21, R: 8, G: 9, B: 8
Clear: 23, R: 9, G: 9, B: 8
Clear: 25, R: 9, G: 10, B: 9
Clear: 27, R: 10, G: 11, B: 10
Clear: 29, R: 11, G: 12, B: 10
Clear: 31, R: 12, G: 13, B: 11
Clear: 33, R: 12, G: 13, B: 12
Clear: 35, R: 13, G: 14, B: 12
Clear: 37, R: 14, G: 15, B: 13
Clear: 39, R: 15, G: 16, B: 14
Clear: 41, R: 15, G: 17, B: 14
Clear: 43, R: 16, G: 17, B: 15
Clear: 45, R: 17, G: 18, B: 16
Clear: 47, R: 17, G: 19, B: 17
Clear: 49, R: 18, G: 20, B: 17
Clear: 51, R: 19, G: 21, B: 18
Clear: 53, R: 20, G: 21, B: 19
Clear: 53, R: 20, G: 22, B: 19
Clear: 55, R: 20, G: 22, B: 19
Clear: 59, R: 22, G: 24, B: 21
Clear: 59, R: 22, G: 24, B: 21
Clear: 61, R: 23, G: 25, B: 21
Clear: 63, R: 23, G: 25, B: 23
Clear: 65, R: 24, G

In [28]:
print("Settings applied. Stabilizing for 1 second...")
time.sleep(1.0) 
print("Stabilizing done, now testing...")
print(" ")

clear_channel_samples = []
for i in range(1000):
    colors = read_colors()
    
    if colors:
        # Save the 'Clear' channel for statistical analysis
        clear_channel_samples.append(colors['clear'])
        
        # Print every 100th sample to avoid flooding the output window
        if i % 100 == 0:
            print(f"Sample {i}: Clear: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

#ANALYSIS
if len(clear_channel_samples) > 0:
    mean_val = np.mean(clear_channel_samples)
    std_dev = np.std(clear_channel_samples)
    noise_percent = (std_dev / mean_val) * 100
    
    print("\n" + "="*40)
    print(f"ANALYSIS RESULT (N=1000)")
    print(f"Mean Count:       {mean_val:.2f}")
    print(f"Standard Dev:     {std_dev:.2f}")
    print(f"Noise (Jitter):   {noise_percent:.3f}%")
    print("="*40)
else:
    print("Error: No data collected.")

Settings applied. Stabilizing for 1 second...
Stabilizing done, now testing...
 
Sample 0: Clear: 321, R: 119, G: 131, B: 112
Sample 100: Clear: 322, R: 120, G: 131, B: 113
Sample 200: Clear: 323, R: 120, G: 131, B: 113
Sample 300: Clear: 324, R: 120, G: 131, B: 113
Sample 400: Clear: 324, R: 120, G: 132, B: 113
Sample 500: Clear: 324, R: 120, G: 132, B: 113
Sample 600: Clear: 323, R: 120, G: 131, B: 113
Sample 700: Clear: 322, R: 120, G: 131, B: 112
Sample 800: Clear: 322, R: 120, G: 131, B: 113
Sample 900: Clear: 322, R: 119, G: 131, B: 113

ANALYSIS RESULT (N=1000)
Mean Count:       322.82
Standard Dev:     0.86
Noise (Jitter):   0.265%


In [ ]:
print(colors)
print(clear_channel_samples)

{'clear': 21, 'red': 13, 'green': 6, 'blue': 4}
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,